In [105]:
from IPython import get_ipython;   
get_ipython().magic('reset -sf')
import pandas as pd
pd.set_option('display.max_colwidth', None)
from IPython.display import display, HTML
import re
import math
from itertools import count
class text_format:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
data = pd.read_excel(r'C:\Users\Mewtwo\Downloads\RFNSA Scrapper.xlsx')



df = pd.DataFrame(data, columns=['Antenna ID No','Antenna Model','Sector','Height - Phase Centre (m)'
                                 ,'Bearing Degrees (true)',
                                 'Mech Downtilt','Elect Downtilt','System',
                                 'Port Number (Band Power per Port (dBm))','Band Power per Port (dBm)'])
df=df.rename(columns={'Antenna Model': 'Antenna', 'Antenna ID No': 'ID', 'Bearing Degrees (true)': 'Bearing'
                  , 'Mech Downtilt': 'MDT', 'Elect Downtilt': 'EDT', 'Height - Phase Centre (m)': 'Height', 
                   'Port Number (Band Power per Port (dBm))': 'Possible Ports', 
                   'Band Power per Port (dBm)': 'Powers'})
df['System'] = df['System'].str.replace(' \[Macro\]', '', regex=True)

#SystemUnique = list( dict.fromkeys(df['System']) )
df['Carrier'] = df['System'].str.split(" /").str[0]
df['tech'] = df['System'].str.split(" / ").str[1:]

df.sort_values(by=['Carrier','ID'], ascending=[True, True], inplace = True)
df['Sector'] = df['Sector'].astype(str).replace('\.0', '', regex=True)

#df = df.reset_index(drop=True)
#df.index += 1

#display( HTML( df.to_html().replace("\\n","<br><li type='1'>") ) )


#df['System'] = df['System'].str.replace(' Microwave Link', 'Dish')
df['EDT'] = df['EDT'].str.replace(' to ', '-', regex=True)
df['EDT'] = df['EDT'].str.replace(r'\(.*\)','', regex=True)
df['Powers'] = df['Powers'].str.replace('49', '79.4 W')
df['Powers'] = df['Powers'].str.replace('48', '63.1 W')
df['Powers'] = df['Powers'].str.replace('47', '50.1 W')
df['Powers'] = df['Powers'].str.replace('46', '39.8 W')
df['Powers'] = df['Powers'].str.replace('45', '31.6 W')
df['Powers'] = df['Powers'].str.replace('44', '25.1 W')
df['Powers'] = df['Powers'].str.replace('43', '20 W')
df['Powers'] = df['Powers'].str.replace('42', '15.8 W')
#df['Sector']= [''.join(f'{m}'+'- Sector ' +f'{l}') for l, m in zip(df['Sector'],df['tech'])]
df['Sector']= [''.join(''.join(map(str, m))+' - Sector ' +f'{l}') for l, m in zip(df['Sector'],df['tech'])]


df.fillna(value = 0,inplace = True)

In [106]:
####Where to add powers?########
Mylist =[]

for i in df['Powers']:
    count = 1
    lister = []
    if isinstance(i, str):
        for j in i:
            if j==';':
                count+=1
            else:
                lister.append(count)
        Mylist.append(lister)
    else:
        Mylist.append(list(str(i))) 
        
    
#display(Mylist)
new_k = []
for elem in Mylist:
    kiterator=[]
    for el in elem:
        if el not in kiterator:
            kiterator.append(el)
    new_k.append(kiterator)
k = new_k

df['WhereToAddPower'] = k
#display(k)

In [107]:
###############Wat should the power be?######
df['Powerss'] = df['Powers'].str.split(";")

#UnformatedPowers = pd.DataFrame()
#UnformatedPowers['UnformatedPowers'] = df['Powers'].values

itArr2 = []
for index, elm in df['Powerss'].iteritems():
    if isinstance(elm, (list, tuple)):
        elm = [sub.replace('W', '') for sub in elm]
        itArr2.append([x for x in elm if x != ''])
    else:
        itArr2.append([elm])
        
df['UnformatedPowers'] = itArr2


#display(df['UnformatedPowers'])
df['Power (W)'] = ['+'.join(map(str, l)) for l in df['UnformatedPowers']]



In [108]:
####Formatting Powers Column with ports numbers

#itArr=[]
#for index, elm in df['Powers'].iteritems():
#    if isinstance(elm, (list, tuple)):
#        itArr.append([f'{i+1}. {l}' for i, l in enumerate(elm)])
#    else:
#        itArr.append([elm])
#df=df.drop(columns='Powers', axis=1)
#df.insert(2, 'Powers', itArr)
#############################################



# Add these Antennas to prox5

In [109]:
AntennasUnique = list( dict.fromkeys(df['Antenna']) )
print('You need to add \n\n'+str.join(" \n", AntennasUnique)+' \n\nto PROX5')

You need to add 

SB1-220-MDL 
RRV4-65D-R6 
AEQP 
AENB 
RR2VV-6533D-R6 
AEQE 

to PROX5


In [110]:
IDUnique = list( dict.fromkeys(df['ID'],df['Antenna']) )
#display(IDUnique)

# IDs for Antennas

In [111]:
AntennaIds = {}
for antenna in AntennasUnique:
    IdsForAntenna = list(dict.fromkeys(df.loc[df['Antenna'] == antenna, 'ID']))
    AntennaIds[antenna] = IdsForAntenna

for x, y in AntennaIds.items():
    print(f'\nThe id of '+text_format.BOLD+ x + text_format.END + ' are\n'+str.join(" \n", y))


The id of SB1-220-MDL are
A4

The id of RRV4-65D-R6 are
12-O 
22-O 
32-O

The id of AEQP are
14-O 
24-O 
34-O

The id of AENB are
16-O 
26-O 
36-O

The id of RR2VV-6533D-R6 are
11-V 
21-V 
31-V

The id of AEQE are
13-V 
23-V 
33-V


# Antenna Settings

In [112]:
Settings = {}
for i in IDUnique:
    IdsForAntenna = list(dict.fromkeys(df.loc[df['ID'] == i, 'ID']))
    AntennaIds[antenna] = IdsForAntenna

for x, y in AntennaIds.items():
    print(f'\nThe id of {x} are\n'+str.join(" \n", y))


The id of SB1-220-MDL are
A4

The id of RRV4-65D-R6 are
12-O 
22-O 
32-O

The id of AEQP are
14-O 
24-O 
34-O

The id of AENB are
16-O 
26-O 
36-O

The id of RR2VV-6533D-R6 are
11-V 
21-V 
31-V

The id of AEQE are
33-V


# Add power to ports

In [114]:
#for i in IDUnique:
#        print(text_format.BOLD+i)
        #display(df.loc[(df['ID'] == i),['ID','System','Powers','Possible Ports']])
display(df[['ID','tech','Possible Ports','WhereToAddPower','Power (W)']])

,ID,tech,Possible Ports,WhereToAddPower,Power (W)
48,A4,[Microwave Link],0,[0],nan
0,12-O,[WCDMA900],1;2;3;4,[3],39.8
9,12-O,[NR/LTE700],1;2;3;4,"[1, 2, 3, 4]",63.1 +63.1 +63.1 +63.1
12,12-O,[NR/LTE900],1;2;3;4,"[1, 2]",63.1 +63.1
15,12-O,[LTE1800],5;6;7;8;9;10;11;12,"[7, 8, 11, 12]",50.1 +50.1 +50.1 +50.1
18,12-O,[NR/LTE2100],5;6;7;8;9;10;11;12,"[7, 8, 11, 12]",50.1 +50.1 +50.1 +50.1
21,12-O,[LTE2600],5;6;7;8;9;10;11;12,"[5, 6, 9, 10]",50.1 +50.1 +50.1 +50.1
24,12-O,[LTE2300],5;6;7;8;9;10;11;12,"[5, 6, 9, 10]",31.6 +31.6 +31.6 +31.6
3,14-O,[NR3500],1;2,[1],79.4
6,16-O,[NR2300],1;2,[1],63.1


# Set Powers

In [38]:
for i in IDUnique:
        print(text_format.BOLD+i)
        display(df.loc[(df['ID'] == i),['ID','System','Powers','Possible Ports']])


A4


,ID,System,Powers,Possible Ports
1,A4,0.0,[nan],0


12-O


,ID,System,Powers,Possible Ports
2,12-O,0.0,"[1. , 2. , 3. 39.8 W, 4. , 5. , 6. , 7. , 8. , 9. , 10. , 11. , 12. ]",1;2;3;4
3,12-O,0.0,"[1. 63.1 W, 2. 63.1 W, 3. 63.1 W, 4. 63.1 W, 5. , 6. , 7. , 8. , 9. , 10. , 11. , 12. ]",1;2;3;4
4,12-O,0.0,"[1. 63.1 W, 2. 63.1 W, 3. , 4. , 5. , 6. , 7. , 8. , 9. , 10. , 11. , 12. ]",1;2;3;4
5,12-O,0.0,"[1. , 2. , 3. , 4. , 5. , 6. , 7. 50.1 W, 8. 50.1 W, 9. , 10. , 11. 50.1 W, 12. 50.1 W]",5;6;7;8;9;10;11;12
6,12-O,0.0,"[1. , 2. , 3. , 4. , 5. , 6. , 7. 50.1 W, 8. 50.1 W, 9. , 10. , 11. 50.1 W, 12. 50.1 W]",5;6;7;8;9;10;11;12
7,12-O,0.0,"[1. , 2. , 3. , 4. , 5. 50.1 W, 6. 50.1 W, 7. , 8. , 9. 50.1 W, 10. 50.1 W, 11. , 12. ]",5;6;7;8;9;10;11;12
8,12-O,0.0,"[1. , 2. , 3. , 4. , 5. 31.6 W, 6. 31.6 W, 7. , 8. , 9. 31.6 W, 10. 31.6 W, 11. , 12. ]",5;6;7;8;9;10;11;12


14-O


...

,ID,System,Powers,Possible Ports
42,23-V,0.0,"[1. 50.1 W, 2. ]",1;2


31-V


,ID,System,Powers,Possible Ports
43,31-V,0.0,"[1. 50.1 W, 2. 50.1 W, 3. 50.1 W, 4. 50.1 W, 5. , 6. , 7. , 8. , 9. , 10. , 11. , 12. ]",1;2;3;4
44,31-V,0.0,"[1. , 2. , 3. , 4. , 5. 50.1 W, 6. 50.1 W, 7. 50.1 W, 8. 50.1 W, 9. 50.1 W, 10. 50.1 W, 11. 50.1 W, 12. 50.1 W]",5;6;7;8;9;10;11;12
45,31-V,0.0,"[1. 50.1 W, 2. 50.1 W, 3. 50.1 W, 4. 50.1 W, 5. , 6. , 7. , 8. , 9. , 10. , 11. , 12. ]",1;2;3;4
46,31-V,0.0,"[1. , 2. , 3. 50.1 W, 4. 50.1 W, 5. , 6. , 7. , 8. , 9. , 10. , 11. , 12. ]",1;2;3;4
47,31-V,0.0,"[1. , 2. , 3. , 4. , 5. 50.1 W, 6. 50.1 W, 7. 50.1 W, 8. 50.1 W, 9. 50.1 W, 10. 50.1 W, 11. 50.1 W, 12. 50.1 W]",5;6;7;8;9;10;11;12
48,31-V,0.0,"[1. , 2. , 3. 15.8 W, 4. 15.8 W, 5. , 6. , 7. , 8. , 9. , 10. , 11. , 12. ]",1;2;3;4


33-V


,ID,System,Powers,Possible Ports
49,33-V,0.0,"[1. 50.1 W, 2. ]",1;2


# EMEG Equipment List

In [39]:

df['tech']= str(df['System'])+ ' - Sector ' + str(df['Sector'])


itArr2 = []
for index, elm in UnformatedPowers['UnformatedPowers'].iteritems():
    if isinstance(elm, (list, tuple)):
        elm = [sub.replace('W', '') for sub in elm]
        itArr2.append([x for x in elm if x != ''])
    else:
        itArr2.append([elm])
        
UnformatedPowers = UnformatedPowers.drop(columns='UnformatedPowers', axis=1)
UnformatedPowers.insert(0, 'UnformatedPowers', itArr)


In [380]:


UnformatedPowers['ID'] = df['ID'].values
UnformatedPowers['EDT'] = df['EDT'].values
UnformatedPowers['tech'] = df['tech'].values
UnformatedPowers['Possible Ports'] = df['Possible Ports'].values

UnformatedPowers['Power (W)'] = ['+'.join(map(str, l)) for l in UnformatedPowers['UnformatedPowers']]

display(UnformatedPowers)

,UnformatedPowers,ID,EDT,tech,Possible Ports,Power (W)
0,[nan],A4,0,3GIS /Dish - Sector 0,0,nan
1,[39.8 ],12-O,2-12,WCDMA900 - Sector 1,1;2;3;4,39.8
2,"[63.1 , 63.1 , 63.1 , 63.1 ]",12-O,2-12,NR/LTE700 - Sector 1,1;2;3;4,63.1 +63.1 +63.1 +63.1
3,"[63.1 , 63.1 ]",12-O,2-12,NR/LTE900 - Sector 1,1;2;3;4,63.1 +63.1
4,"[50.1 , 50.1 , 50.1 , 50.1 ]",12-O,2-12,LTE1800 - Sector 1,5;6;7;8;9;10;11;12,50.1 +50.1 +50.1 +50.1
5,"[50.1 , 50.1 , 50.1 , 50.1 ]",12-O,2-12,NR/LTE2100 - Sector 1,5;6;7;8;9;10;11;12,50.1 +50.1 +50.1 +50.1
6,"[50.1 , 50.1 , 50.1 , 50.1 ]",12-O,2-12,LTE2600 - Sector 1,5;6;7;8;9;10;11;12,50.1 +50.1 +50.1 +50.1
7,"[31.6 , 31.6 , 31.6 , 31.6 ]",12-O,2-12,LTE2300 - Sector 1,5;6;7;8;9;10;11;12,31.6 +31.6 +31.6 +31.6
8,[79.4 ],14-O,6,NR3500 - Sector 1,1;2,79.4
9,[63.1 ],16-O,7,NR2300 - Sector 1,1;2,63.1
